In [201]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [202]:
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

In [203]:
# Define the data path
data_path = '/kaggle/input/metal-glassgarbage-classification-data/Garbage classification'
# List the classes in the data directory
all_classes = os.listdir(data_path)
if 'trash' in all_classes:
    all_classes.remove('trash')
print(all_classes)



['metal', 'glass', 'paper', 'cardboard', 'plastic']


In [204]:

# Data generators with preprocessing for Xception
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=45,
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range=0.3,
    zoom_range=0.25,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.4, # Define validation split directly in datagen
    brightness_range=[0.8, 1.2],  # Added brightness augmentation
    channel_shift_range=20.0     # Added channel shift range for color variations
)
def filter_directory(data_path, valid_classes):
    # Return only the directories that match valid_classes
    return [dir for dir in os.listdir(data_path) if dir in valid_classes]
    
valid_dirs = filter_directory(data_path, all_classes)

# Training data generator
data_train = datagen.flow_from_directory(
    str(data_path),
    target_size=(224, 224),  # Image size for Xception
    batch_size=32,
    class_mode='categorical',
    subset='training' , # Specify subset as 'training'
    classes=valid_dirs  # Use the filtered valid directories

)


# Validation data generator
data_val = datagen.flow_from_directory(
    str(data_path),
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation' , # Specify subset as 'validation'
    classes=valid_dirs  # Use the filtered valid directories

)

Found 1436 images belonging to 5 classes.
Found 954 images belonging to 5 classes.


In [205]:
# Load Xception base model without the top layer
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model so that we don't train it initially
base_model.trainable = False

# Check the architecture of the base model
base_model.summary()


Model: "xception"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 111, 111,  │        864 │ input_layer_18[0… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 111, 111,  │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 111, 111,  │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 109, 109,  │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 109, 109,  │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 109, 109,  │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 109, 109,  │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 109, 109,  │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 109, 109,  │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 109, 109,  │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 109, 109,  │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_72 (Conv2D)  │ (None, 55, 55,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 55, 55,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 55, 55,    │        512 │ conv2d_72[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_216 (Add)       │ (None, 55, 55,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 55, 55,    │          0 │ add_216[0][0]   

 Total params: 20,861,480 (79.58 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 20,861,480 (79.58 MB)

In [206]:
# for layer in base_model.layers[:-10]:  # Freeze all layers except the last 10
#     layer.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduce dimensionality
# x = Dropout(0.4)(x)              # Prevent overfitting
x = Dense(512, activation='relu')(x)  # Fully connected layer
output_layer = Dense(5, activation='softmax', dtype='float32')(x)  # Force float32 output for stability

# Create the full model
model = Model(inputs=base_model.input, outputs=output_layer)


In [207]:
# Callbacks for optimization
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    min_lr=1e-6,
    verbose=1
)

checkpoint = ModelCheckpoint(
    'best_model.keras',  # Change the file extension to .keras
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)


In [208]:
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=0.0001,
#     decay_steps=100000,
#     decay_rate=0.96,
#     staircase=True
# )
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [209]:
from sklearn.utils import class_weight
# Get the class labels (for computing class weights)
class_indices = data_train.class_indices
num_classes = len(class_indices)

# Convert the class indices to class labels
class_labels = list(class_indices.keys())

# Get the number of samples in each class
class_counts = np.zeros(num_classes)
for i, label in enumerate(class_labels):
    class_counts[i] = sum(data_train.classes == i)

# Compute class weights using the formula
class_weights = class_weight.compute_class_weight(
    'balanced', 
    classes=np.unique(data_train.classes), 
    y=data_train.classes
)

# Convert class weights to a dictionary for easy usage
class_weight_dict = {class_labels[i]: class_weights[i] for i in range(num_classes)}

print("Class weights: ", class_weight_dict)

Class weights:  {'metal': 1.167479674796748, 'glass': 0.9541528239202658, 'paper': 0.8044817927170869, 'cardboard': 1.1867768595041321, 'plastic': 0.9903448275862069}


In [ ]:
class_weights = {0: 1.167479674796748, 1:0.9541528239202658, 2:0.8044817927170869, 3: 1.1867768595041321, 4:0.9903448275862069}  # Adjust based on class distribution

history = model.fit(
    data_train,
    epochs=10,
    validation_data=data_val,
    callbacks=[early_stopping, checkpoint ,reduce_lr],
    class_weight=class_weights
)

for layer in model.layers[-30:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history_fine = model.fit(
    data_train,
    validation_data=data_val,
    epochs=10,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 536ms/step - accuracy: 0.4356 - loss: 1.3789
Epoch 1: val_loss improved from inf to 0.85608, saving model to best_model.keras
45/45 ━━━━━━━━━━━━━━━━━━━━ 63s 1s/step - accuracy: 0.4387 - loss: 1.3734 - val_accuracy: 0.6929 - val_loss: 0.8561 - learning_rate: 1.0000e-04
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step - accuracy: 0.7728 - loss: 0.6766
Epoch 2: val_loss improved from 0.85608 to 0.70361, saving model to best_model.keras
45/45 ━━━━━━━━━━━━━━━━━━━━ 33s 674ms/step - accuracy: 0.7729 - loss: 0.6754 - val_accuracy: 0.7254 - val_loss: 0.7036 - learning_rate: 1.0000e-04
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step - accuracy: 0.8295 - loss: 0.5269
Epoch 3: val_loss improved from 0.70361 to 0.65202, saving model to best_model.keras
45/45 ━━━━━━━━━━━━━━━━━━━━ 33s 668ms/step - accuracy: 0.8293 - loss: 0.5270 - val_accuracy: 0.7411 - val_loss: 0.6520 - learning_rate: 1.0000e-04
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step - accuracy: 0.8

In [ ]:
# Plot training & validation accuracy
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training & validation loss
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Predictions
data_val.reset()
predictions = model.predict(data_val)
y_pred = np.argmax(predictions, axis=1)
y_true = data_val.classes

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=data_val.class_indices.keys(), yticklabels=data_val.class_indices.keys())
plt.title('Confusion Matrix')
plt.ylabel('True Labels')
plt.xlabel('Predicted Labels')
plt.show()


In [ ]:
# Classification Report
print(classification_report(y_true, y_pred, target_names=data_val.class_indices.keys()))


In [ ]:
fpr = {}
tpr = {}
roc_auc = {}
for i in range(data_train.num_classes):
    fpr[i], tpr[i], _ = roc_curve(tf.keras.utils.to_categorical(y_true, data_train.num_classes)[:, i], predictions[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve
plt.figure(figsize=(10, 8))
for i, label in enumerate(data_val.class_indices.keys()):
    plt.plot(fpr[i], tpr[i], label=f'{label} (AUC = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='best')
plt.show()